# Notebook to create a pedestrian model with the narrow corridor measurements

#### Install the physics-based-pedestrian-modeling package
Note: You might need to restart the session after installing the package.

In [ ]:
!pip install physics-based-pedestrian-modeling

#### Importing libraries

In [ ]:
import logging
from pprint import pformat

from omegaconf import OmegaConf

from physped.utils.config_utils import initialize_hydra_config, set_plot_style
from physped.io.readers import trajectory_reader
from physped.preprocessing.trajectories import preprocess_trajectories, process_slow_modes
from physped.core.functions_to_discretize_grid import learn_potential_from_trajectories
from physped.core.trajectory_simulator import simulate_trajectories
from physped.visualization.plot_trajectories import plot_trajectories

#### Configuration and setup

Steps:
- Read the configuration file
- Setup the logging
- Set the matplotlib plot tyle
- Change the working directory

Available configuration files:
- Narrow corridor paths
- Intersecting paths
- Train station platform paths

In [ ]:
# Select one of the available environments
environments = {
    'Narrow corridor paths': 'single_paths',
    'Intersecting paths': 'intersecting_paths',
    'Train station platform paths': 'asdz_pf12'
}
env_name = environments.get('Narrow corridor paths')

# Initialize Hydra configuration
config = initialize_hydra_config(env_name)

# set logging level
log = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO, force = True)

# set plot style
set_plot_style(use_latex=False)

The default model parameters are set to:
- Noise intensiy $\sigma = 0.9\, \textrm{m s}^{-3/2}$
- Relaxation time $\tau = 0.5$ s
- Integration timestep $\Delta t=\frac{1}{15}$

In [ ]:
logging.info(
    "MODELING PARAMETERS: \n" + 
    pformat(OmegaConf.to_container(config.params.model, resolve=True), depth=1)
    )

#### Optional: overwriting the model parameters
Any parameter in the config can be overwritten like demonstrated in the cell below

In [ ]:
overwrite_config_params = False
if overwrite_config_params:
    config.params.model.sigma = 0.9
    config.params.model.taux = 0.5
    config.params.model.tauu = 0.5
    config.params.model.dt = 1/config.params.model.fps

#### Read the raw trajectories 
In this notebook the trajectories are read from a remote repository on 4TU.

In [ ]:
import requests
link = "https://data.4tu.nl/ndownloader/items/b8e30f8c-3931-4604-842a-77c7fb8ac3fc/versions/1"
response = requests.get(link, timeout=10)
response.status_code

In [ ]:
response.content

In [ ]:
trajectories = trajectory_reader[env_name](config)
logging.info('\n' + pformat(trajectories.head()))

#### Preprocess the trajectories.
Steps:
- Rename the columns
- Prune short trajectories
- Add a trajectory index
- Compute velocities in Cartesian coordinates
- Convert velocities to polar coordinates

In [ ]:
trajectories = preprocess_trajectories(trajectories, config=config)

#### Compute the slow dynamics.
Steps:
- Compute the slow velocities $\vec{u}_s$
- Compute the slow positions $\vec{x}_s$

In [ ]:
trajectories = process_slow_modes(trajectories, config)

#### Plot the preprocessed trajectories

In [ ]:
plot_trajectories(trajectories, config, "recorded")

#### Compute the piecewise potential
Steps:
- Create the lattice to discretize the slow dynamics
- Fit the actual dynamics conditioned to the slow dynamics with Gaussian distributions

In [ ]:
reduce_lattice_size = True
if reduce_lattice_size:
    config.params.grid.r.list = [0, 0.5, 1, 1.5, 2, 2.5]
    config.params.grid.spatial_cell_size = 0.4
piecewise_potential = learn_potential_from_trajectories(trajectories, config)

#### Simulate new trajectories with the model
- Sample initial conditions from the trajectory measurements
- Integrate the generalized pedestrian model

The number of trajectories to simulate can be overwritten in the configuration as shown below. 

In [ ]:
config.params.simulation.ntrajs = 25
simulated_trajectories = simulate_trajectories(piecewise_potential, config, trajectories)

#### Plot the simulated trajectories

In [ ]:
plot_trajectories(simulated_trajectories, config, "simulated")